# side model feature selection

In [1]:
import pandas as pd

df_features = pd.read_parquet("data/features.parquet")
print(df_features.shape)
df_features.isna().sum(axis=0).sort_values(ascending=False)

(13219, 12127)


adx_7                     0
cwt_win1024_12_dt_lag5    0
cwt_win1024_3_dt_lag5     0
cwt_win1024_4_dt_lag5     0
cwt_win1024_5_dt_lag5     0
                         ..
cwt_win128_20_lag12       0
cwt_win128_0_lag13        0
cwt_win128_1_lag13        0
cwt_win128_2_lag13        0
williams_r_ddt_lag19      0
Length: 12127, dtype: int64

In [2]:
import numpy as np

label = np.load("data/side_label.npy")

print(label.shape)

len_gap = len(label) - len(df_features)

label = label[len_gap:]

print(label.shape)

(15059,)
(13219,)


In [3]:
from custom_indicators.toolbox.feature_selection.catfcq_selector import CatFCQSelector

selector = CatFCQSelector()
selector.fit(df_features, label)

➤ 识别数值型变量...
➤ 计算特征与目标变量的相关性(使用随机森林)...
✓ 选择第1个特征: cmma (最大重要性: 5.3205)
➤ 计算特征冗余度...
➤ 总计选择2425个特征 (已选择1个，还需选择2424个)...
➤ 开始MRMR迭代选择过程...


选择特征:   0%|                                                          | 0/2424 [00:00<?, ?特征/s]


✅ 特征选择完成：从12127个特征中选择了2425个，舍弃了9702个
✅ 选择的特征: ['cmma', 'adaptive_bp_dt_lag14', 'change_variance_ratio_ddt_lag16', 'ac_46', 'change_variance_ratio_ddt_lag11', 'entropy_for_jesse_ddt_lag14', 'natr_dt', 'natr', 'hurst_coef_fast_ddt_lag8', 'williams_r_ddt_lag2', 'phase_accumulation_lag1', 'ehlers_early_onset_trend_ddt_lag4', 'vwap_dt', 'comb_spectrum_dom_cycle_ddt_lag12', 'williams_r', 'sample_entropy_win32_array', 'pfe_ddt_lag17', 'hurst_coef_slow_lag17', 'dual_diff_dt_lag16', 'dft_spectrum_5', 'fti_ddt_lag3', 'hurst_coef_slow_ddt', 'cwt_win32_7_ddt_lag14', 'adx_14_ddt_lag3', 'dft_dom_cycle_dt_lag5', 'cwt_win128_2_dt_lag1', 'roll_measure_lag8', 'fisher_dt', 'dft_dom_cycle_ddt_lag9', 'vmd_win64_1_ddt_lag3', 'hurst_coef_slow_dt_lag19', 'vmd_win128_0_lag3', 'vmd_win256_2_lag14', 'cwt_win32_3_ddt_lag12', 'bekker_parkinson_vol', 'adx_7_ddt_lag9', 'cwt_win32_0_ddt_lag18', 'acp_pwr_38', 'trendflex_dt', 'evenbetter_sinewave_short_ddt_lag13', 'adx_14_lag18', 'fti_best_period_ddt_lag6', 'approxima

In [4]:
side_res_long = pd.Series(selector.relevance_, index=selector.variables_).sort_values(
    ascending=False
)
print((side_res_long > 0).sum())
side_res_long

3355


cmma                5.320455
williams_r          2.109272
natr_dt             2.012565
natr                1.948791
vwap_dt             1.382357
                      ...   
cwt_win256_5_ddt    0.000000
cwt_win256_6_ddt    0.000000
cwt_win256_7_ddt    0.000000
cwt_win256_8_ddt    0.000000
cwt_win32_7_dt      0.000000
Length: 12127, dtype: float64

In [5]:
import json

feature_info = {
    "side": {
        "side": [],
    },
    "meta": {
        "meta": [],
        "model": [],
    },
}

feature_long = side_res_long[side_res_long > 0].index.tolist()


feature_info["side"]["side"] = feature_long


with open("strategies/BinanceBtcEntropyBarV1/feature_info.json", "w") as f:
    json.dump(feature_info, f, indent=4)

# meta model feature selection

In [1]:
import numpy as np
import pandas as pd

df_features = pd.read_parquet("data/features.parquet")
meta_label = np.load("data/label_meta.npy")
print(df_features.shape)
print(meta_label.shape)

df_features.isna().sum(axis=0).sort_values(ascending=False)

(13219, 12127)
(13219,)


adx_7                     0
cwt_win1024_12_dt_lag5    0
cwt_win1024_3_dt_lag5     0
cwt_win1024_4_dt_lag5     0
cwt_win1024_5_dt_lag5     0
                         ..
cwt_win128_20_lag12       0
cwt_win128_0_lag13        0
cwt_win128_1_lag13        0
cwt_win128_2_lag13        0
williams_r_ddt_lag19      0
Length: 12127, dtype: int64

In [2]:
from strategies.BinanceBtcEntropyBarV1.config import (
    SIDE,
    get_side_model,
)

side_model = get_side_model(False)

side_res = side_model.predict(df_features[SIDE])

assert df_features.shape[0] == len(side_res)

df_features["model"] = side_res

In [3]:
# meta feature selection
from custom_indicators.toolbox.feature_selection.catfcq_selector import CatFCQSelector

selector = CatFCQSelector()
selector.fit(df_features, meta_label)
meta_res = pd.Series(selector.relevance_, index=selector.variables_).sort_values(
    ascending=False
)

➤ 识别数值型变量...
➤ 计算特征与目标变量的相关性(使用随机森林)...
✓ 选择第1个特征: model (最大重要性: 6.7817)
➤ 计算特征冗余度...
➤ 总计选择2425个特征 (已选择1个，还需选择2424个)...
➤ 开始MRMR迭代选择过程...


选择特征:   0%|                                                          | 0/2424 [00:00<?, ?特征/s]


✅ 特征选择完成：从12128个特征中选择了2425个，舍弃了9703个
✅ 选择的特征: ['model', 'voss_filt_ddt_lag7', 'approximate_entropy_win128_spot', 'dft_spectrum_13', 'cwt_win1024_0_ddt_lag18', 'sample_entropy_win256_array', 'stc_ddt_lag19', 'ac_29', 'vmd_win1024_0_lag8', 'phase_accumulation_ddt_lag19', 'sample_entropy_win256_spot', 'cmma', 'fti_ddt_lag6', 'cwt_win512_0_ddt_lag13', 'comb_spectrum_pwr_4', 'bekker_parkinson_vol_ddt_lag14', 'stc_ddt_lag18', 'sample_entropy_win64_array', 'hurst_coef_fast_ddt_lag19', 'approximate_entropy_win256_spot', 'vmd_win32_0_lag1', 'dft_spectrum_9', 'hurst_coef_slow_ddt_lag11', 'comb_spectrum_dom_cycle_ddt_lag5', 'dft_spectrum_3', 'price_change_oscillator_dt_lag1', 'williams_r', 'vmd_win128_1_ddt_lag8', 'fti_dt_lag18', 'entropy_for_jesse_lag10', 'phase_accumulation_dt_lag19', 'dual_diff_lag5', 'iqr_ratio_lag8', 'ac_37', 'stc_dt', 'phase_accumulation_lag18', 'cwt_win32_7_ddt_lag14', 'acp_pwr_0', 'homodyne_lag17', 'stc_ddt_lag17', 'norm_on_balance_volume_lag2', 'cwt_win32_4_ddt_lag8', '

In [6]:
(meta_res > 0).sum()

2664

In [7]:
import json
from pathlib import Path

feature_info_path = Path("strategies/BinanceBtcEntropyBarV1/feature_info.json")
if feature_info_path.exists():
    with open(feature_info_path, "r") as f:
        feature_info = json.load(f)

feature_info["meta"]["meta"] = []
feature_info["meta"]["model"] = []

meta_features = meta_res[meta_res > 0].index.tolist()
for k in meta_features:
    if k == "model":
        feature_info["meta"]["model"].append(k)
    else:
        feature_info["meta"]["meta"].append(k)


with open("strategies/BinanceBtcEntropyBarV1/feature_info.json", "w") as f:
    json.dump(feature_info, f, indent=4)